In [1]:
import torch.optim as optim
from torch.utils.data import DataLoader
import sys

sys.path.append('..')
from shared.models import *
from shared.new_datasets import *


def metrics(true_positive, total_truth, predicted_positive, correct_total, total):
    n_way = len(true_positive) # Retrieve n_way from the length of the variables. All 3 inputs should be the same length
    f1_flag = 0  # Flag for invalid F1 score
    precision = list(0. for i in range(n_way))
    recall = list(0. for i in range(n_way))
    class_f1 = list(0. for i in range(n_way))
    
    # Find class accuracy, precision and recall
    for j in range(n_way):
        if (predicted_positive[j] != 0 and true_positive[j] != 0):  # Check if F1 score is valid
            precision[j] = true_positive[j] / predicted_positive[j]
            recall[j] = true_positive[j] / total_truth[j]  # Recall is the same as per class accuracy
            class_f1[j] = 2 * precision[j] * recall[j] / (precision[j] + recall[j])
        else:
            f1_flag = 1

    # Find Accuracy, Macro Accuracy and Macro F1 Score
    macro_acc_sum = 0
    f1_sum = 0
    for k in range(n_way):
        macro_acc_sum += recall[k]
        if f1_flag == 0:  # Check for invalid f1 score
            f1_sum += class_f1[k]

    accuracy = correct_total / total
    macro_accuracy = macro_acc_sum / n_way
    f1_score = f1_sum / n_way
    return accuracy, macro_accuracy, f1_score, class_f1


def train(inputs, labels, model, criterion, device, optimizer, freeze=False):
    # Training loop
    model.train()
    
    # Freeze all layers except those indicated
    if freeze:
        for name, param in model.named_parameters():
            if name not in freeze:
                param.requires_grad = False

    # Train the entire support set in one batch
    optimizer.zero_grad()
    pred = model(inputs)
    loss = criterion(pred, labels)
    loss.backward()
    optimizer.step()
    train_loss = loss.item()  # Running training loss
    
    return train_loss
    

def test(inputs, labels, model, criterion, device, n_way):
    # An F1 Score of 0 indicates that it is invalid
    model.eval()
    true_positive = list(0. for i in range(n_way))  # Number of correctly predicted samples per class
    total_truth = list(0. for i in range(n_way))  # Number of ground truths per class
    predicted_positive = list(0. for i in range(n_way))  # Number of predicted samples per class
    correct_total = 0  # Total correctly predicted samples
    total = 0  # Total samples
    with torch.no_grad():
        # Test the entire query set in one batch
        pred = model(inputs)
        loss = criterion(pred, labels)
        val_loss = loss.item()  # Running validation loss
        _, predicted = torch.max(pred, 1)
        correct = (predicted == labels).squeeze()  # Samples that are correctly predicted
        correct_total += (predicted == labels).sum().item()
        total += labels.size(0)

        for i in range(len(predicted)):
            label = labels[i]
            true_positive[label] += correct[i].item()
            total_truth[label] += 1
            predicted_positive[predicted[i].item()] += 1  # True Positive + False Positive
            
    accuracy, macro_accuracy, f1_score, class_f1 = metrics(true_positive, total_truth,
                                                           predicted_positive, correct_total, total)
    
    return val_loss, accuracy, macro_accuracy, f1_score, class_f1

In [2]:
#def main():
# Set Training Parameters
n_way = 3
k_shot = 5
k_query = 16
num_episodes = 5
num_epochs = 100
num_workers = 12
bs = 16
lr = 1e-4
root = '../../../../scratch/rl80/mimic-cxr-jpg-2.0.0.physionet.org/files'
path_splits = '../splits/splits.csv'  # Location of preprocessed splits
path_results = '../../results'  # Folder to save the CSV results
path_pretrained = '../results/basic/basic_36.pth'
freeze = ['linear.weight', 'linear.bias']  # Freeze all layers except linear layers

torch.cuda.set_device(0)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
criterion = nn.CrossEntropyLoss()

# Load in data
dataset = NovelMimicCxrJpg(root, path_splits, n_way, k_shot, k_query, num_episodes)
loader = DataLoader(dataset, batch_size=bs, shuffle=True, num_workers=num_workers)

# Create Dataframe to export results to CSV
df_results = pd.DataFrame(columns=['Epoch', 'Training Loss', 'Validation Loss', 'Accuracy', 'Macro Accuracy',
                                   'Macro-F1 Score'] + [str(x) + ' F1' for x in range(n_way)])

# Iterate through batched episodes. One episode is one experiment
for step, (support_imgs, support_labels, query_imgs, query_labels) in enumerate(loader):
    # Convert Tensors to appropriate device
    batch_support_x, batch_support_y = support_imgs.to(device), support_labels.to(device)
    batch_query_x, batch_query_y = query_imgs.to(device), query_labels.to(device)

    # [num_batch, training_sz, channels, height, width] = support_x.size()
    # num_batch = num of episodes
    # training_sz = size of support or query set
    num_batch = batch_support_x.size(0) # Number of episodes in the batch
    
    # Break down the batch of episodes into single episodes
    for i in range(num_batch):
        # Load in model and reset weights every episode/experiment
        model = BaselineNet(n_way).to(device)
        pretrained_dict = torch.load(path_pretrained)
        del pretrained_dict['linear.weight']  # Remove the last linear layer
        del pretrained_dict['linear.bias']
        model_dict = model.state_dict()
        model_dict.update(pretrained_dict)
        model.load_state_dict(model_dict)
        
        # Reset optimizer with model parameters
        optimizer = optim.Adam(model.parameters(), lr=lr)
        
        # Break down the sets into individual episodes
        support_x, support_y = batch_support_x[i], batch_support_y[i]
        query_x, query_y = batch_query_x[i], batch_query_y[i]

        # Variables for best epoch per experiment
        best_score = 0
        best_epoch = 0
        df_best = pd.DataFrame(columns=['Epoch', 'Training Loss', 'Validation Loss', 'Accuracy', 'Macro Accuracy',
                                   'Macro-F1 Score'] + [str(x) + ' F1' for x in range(n_way)]) # Track best epoch
        # Training and testing for specified epochs
        for epoch in range(num_epochs):
            # Training
            train_loss = train(support_x, support_y, model, criterion, device, optimizer, freeze=freeze)

            # Testing
            val_loss, acc, m_acc, macro_f1, class_f1 = test(query_x, query_y, model, criterion, device, n_way)
            
            # Find best epoch
            score = 0.5*acc + 0.5*macro_f1
            if score > best_score:
                best_score = score
                best_epoch = epoch + 1
                df_best.loc[0] = [epoch + 1, train_loss, val_loss, acc, m_acc, macro_f1] + class_f1
                
            #print(
            #    f'[{epoch + 1}] t_loss: {train_loss:.5f} v_loss: {val_loss:.5f} val_acc: {acc:.5f} '
            #    f'val_m_acc: {m_acc:.5f} f1: {macro_f1:.5f}')
        
        # Print the best results per experiment
        print(
            f'[{int(df_best.iloc[0,0])}] t_loss: {df_best.iloc[0,1]} v_loss: {df_best.iloc[0,2]} '
            f'val_acc: {df_best.iloc[0,3]} f1: {df_best.iloc[0,5]}')
    
        # Record the best epoch to be saved into a CSV
        df_results = df_results.append(df_best.loc[0], ignore_index=True)
        
# Create results folder if it does not exist
if not os.path.exists(path_results):
    os.makedirs(path_results)
    
# Export results to a CSV file
df_results.to_csv(os.path.join(path_results, f'{k_shot}shot_baseline.csv'), index=False)


#if __name__ == '__main__':
#    main()


[34] t_loss: 0.009635225869715214 v_loss: 1.0729824304580688 val_acc: 0.4583333333333333 f1: 0.4527458492975734
[22] t_loss: 0.043317437171936035 v_loss: 1.2448405027389526 val_acc: 0.4583333333333333 f1: 0.4568627450980392
[15] t_loss: 0.06880378723144531 v_loss: 1.3845996856689453 val_acc: 0.3125 f1: 0.3116883116883117
[19] t_loss: 0.03704823553562164 v_loss: 1.2352722883224487 val_acc: 0.4375 f1: 0.43841642228739003
[78] t_loss: 0.0025841225869953632 v_loss: 1.6353203058242798 val_acc: 0.4166666666666667 f1: 0.39656352834647396


In [53]:
labels = [3, 2, 2, 3, 4, 4, 4, 2, 4, 2, 3, 3, 2, 4, 3]
new_labels = labels
unique_labels = np.unique(labels)
unique_labels

array([2, 3, 4])

In [8]:
df_best

,Epoch,Training Loss,Validation Loss,Accuracy,Macro Accuracy,Macro-F1 Score,0 F1,1 F1,2 F1
0,21.0,0.050457,1.249611,0.520833,0.520833,0.464103,0.6,0.1,0.692308


In [43]:

df_results

,Epoch,Training Loss,Validation Loss,Accuracy,Macro Accuracy,Macro-F1 Score,0 F1,1 F1,2 F1
0,21.0,0.050457,1.249611,0.520833,0.520833,0.464103,0.6,0.1,0.692308
1,21.0,0.050457,1.249611,0.520833,0.520833,0.464103,0.6,0.1,0.692308
2,21.0,0.050457,1.249611,0.520833,0.520833,0.464103,0.6,0.1,0.692308
3,21.0,0.050457,1.249611,0.520833,0.520833,0.464103,0.6,0.1,0.692308
4,21.0,0.050457,1.249611,0.520833,0.520833,0.464103,0.6,0.1,0.692308
5,21.0,0.050457,1.249611,0.520833,0.520833,0.464103,0.6,0.1,0.692308
6,21.0,0.050457,1.249611,0.520833,0.520833,0.464103,0.6,0.1,0.692308
7,21.0,0.050457,1.249611,0.520833,0.520833,0.464103,0.6,0.1,0.692308


In [44]:
os.path.join('test','test')

'test/test'

In [40]:
t5 = [x for x, label in enumerate(labels) if label == 2]